# Load Dataset

In [ ]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [ ]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx.
num_section = 11

dat = alldat[num_section]
print(dat.keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


In [ ]:
print(len(alldat))
print(alldat[0]['response'])

39
[ 1. -1.  1.  0.  1.  1. -1. -1.  0.  1.  1.  0.  1.  1.  1. -1.  0. -1.
 -1. -1.  0.  1.  1.  0.  1.  1.  0. -1.  1. -1. -1.  0. -1.  1. -1.  0.
  0. -1. -1.  1.  1. -1. -1. -1.  0. -1. -1.  1. -1.  1. -1. -1.  0.  1.
  1. -1. -1. -1. -1. -1. -1. -1.  0. -1.  1. -1. -1.  0. -1.  0.  1. -1.
  1. -1. -1.  1. -1. -1. -1.  0.  0.  1. -1. -1.  1.  0.  0.  1. -1.  1.
  1.  1.  1.  0. -1.  1. -1.  0.  1.  0. -1.  1. -1.  0. -1.  0. -1.  0.
 -1.  0.  0.  0.  0.  0. -1.  0.  1.  1.  0.  1.  0.  1. -1.  1. -1.  1.
 -1. -1.  1.  0. -1.  0. -1. -1. -1.  0.  0.  1.  0. -1.  1. -1. -1.  1.
 -1.  0.  0.  1.  0.  1.  1.  0.  1. -1.  0. -1.  1. -1.  0.  0.  0.  1.
  1.  0. -1.  1.  1.  1.  0.  0.  0.  0.  1.  0.  0. -1.  1.  1.  1.  0.
  0.  1.  0.  0.  0. -1.  0. -1.  0.  1.  1.  0.  0.  1.  0.  0.  0.  1.
  0.  0.  1.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  1.]


`alldat` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `dat = alldat[k]`, you have the following fields:

* `dat['mouse_name']`: mouse name
* `dat['date_exp']`: when a session was performed
* `dat['spks']`: neurons by trials by time bins.    
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `dat['contrast_left']`: contrast level for left stimulus. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_time']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `dat['wheel']`: exact position of the wheel that the mice uses to make a response, binned at `10ms`. 
* `dat['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position. 
* `dat['lfp']`: recording of the local field potential in each brain area from this experiment, binned at `10ms`.
* `dat['brain_area_lfp']`: brain area names for the LFP channels. 
* `dat['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons". 
* `dat['waveform_w']`: temporal components of spike waveforms. `w@u` reconstructs the time by channels action potential shape. 
* `dat['waveform_u]`: spatial components of spike waveforms.
* `dat['%X%_passive']`: same as above for `X` = {`spks`, `lfp`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses. 




# Prepare Data

## Data for visual stimulus and response

In [ ]:
response = dat['response'] # right - nogo - left (-1, 0, 1) [340,] [n_trial,]
response_move = np.where(response!=0, 1, 0)  # 1 as have response, 0 as not

vis_right = dat['contrast_right'] # 0 - low - high [340,] [n_trial,]
vis_left = dat['contrast_left'] # 0 - low - high [340,] [n_trial,]
vis_diff = vis_left - vis_right

stim_ipsi = np.where(vis_left!=0, 1, 0)  # ipsilateral stimulus [340,] [n_trial,]
stim_contr = np.where(vis_right!=0, 1, 0)  # contralateral stilumus [340,] [n_trial,]

In [ ]:
# print(stim_ipsi.shape)
# print(vis_left)
# print(stim_ipsi)
# print(response)
# print(response_move)

## Data for spike fragments related with stimulus

In [ ]:
spks = dat['spks']  # [n_neuron, n_trial, n_time] (698, 340, 250)
n_neuron, n_trial, n_time = spks.shape[0], spks.shape[1], spks.shape[2]

idx_spks_stim_start = 44  # idx of starting visual stimulus
idx_spks_stim_end = 84
spks_frag_stim = spks[..., idx_spks_stim_start:idx_spks_stim_end]  # fragment of spikes related with stimulus (698, 340, 20)

In [ ]:
# print(spks.shape)
# print(spks_frag_stim.shape)

In [ ]:
print(dat['response_time'].shape)
# print(dat['response_time'])

(340, 1)


## Data for spike fragments related with response

In [ ]:
response_time = dat['response_time']  # [n_trial,1] (340,1)
idx_response_time = np.round(response_time.squeeze()/0.01).astype(int)  # [n_trial,] (340,)


# fragment of spikes related with response (698, 340, 5)
num_pre = 5  # num of time bin pre the response to collect spks
# spks_frag_response = spks[..., idx_response_time]
for idx_trial in range(n_trial):  # using loop, should have a better way...
    if idx_trial==0:
        spks_frag_response = spks[:, idx_trial, idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial]]
        spks_frag_response = spks_frag_response[:,np.newaxis,:]
    else:
        spks_frag_new = spks[:, idx_trial, idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial]]
        spks_frag_new = spks_frag_new[:,np.newaxis,:]
        spks_frag_response = np.concatenate((spks_frag_response, spks_frag_new), axis=1)

In [ ]:
# print(idx_response_time.shape)
print(spks_frag_response.shape)

(698, 340, 5)


# Model Fitting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

## Logistic reg for spks_frag_stim---stim_ipsi (L)

Investigate the relationship between spike fragment which related to stimulus and mice's left stimulus

In [ ]:

list_score_stimL_train = []
list_score_stimL_test = []
for idx_neuron in range(n_neuron):
    X_stimL = spks_frag_stim[idx_neuron]  # [n_trial, n_time] (340, 250)
    y_stimL = stim_ipsi  # [n_trial,] (340,)

    X_stimL_train, X_stimL_test, y_stimL_train, y_stimL_test = train_test_split(X_stimL, y_stimL, test_size=0.2)
    log_reg_l1L = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_stimL_train, y_stimL_train)

    score_trainL = log_reg_l1L.score(X_stimL_train, y_stimL_train)
    score_testL = log_reg_l1L.score(X_stimL_test, y_stimL_test)
    list_score_stimL_train.append(score_trainL)
    list_score_stimL_test.append(score_testL)

In [ ]:
print(max(list_score_stimL_train))
print(max(list_score_stimL_test))
print('-----------------------------')
print(min(list_score_stimL_train))
print(min(list_score_stimL_test))
print('-----------------------------')
print(sum(list_score_stimL_train)/n_neuron)
print(sum(list_score_stimL_test)/n_neuron)

0.8088235294117647
0.7205882352941176
-----------------------------
0.47058823529411764
0.27941176470588236
-----------------------------
0.5848327153210848
0.4952595651441094


## Logistic reg for spks_frag_stim---stim_contr (R)

Investigate the relationship between spike fragment which related to stimulus and mice's right stimulus

In [ ]:

list_score_stimR_train = []
list_score_stimR_test = []
for idx_neuron in range(n_neuron):
    X_stimR = spks_frag_stim[idx_neuron]  # [n_trial, n_time] (340, 250)
    y_stimR = stim_contr  # [n_trial,] (340,)

    X_stimR_train, X_stimR_test, y_stimR_train, y_stimR_test = train_test_split(X_stimR, y_stimR, test_size=0.2)
    log_reg_l1R = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_stimR_train, y_stimR_train)

    score_trainR = log_reg_l1R.score(X_stimR_train, y_stimR_train)
    score_testR = log_reg_l1R.score(X_stimR_test, y_stimR_test)
    list_score_stimR_train.append(score_trainR)
    list_score_stimR_test.append(score_testR)

In [ ]:
print(max(list_score_stimR_train))
print(max(list_score_stimR_test))
print('-----------------------------')
print(min(list_score_stimR_train))
print(min(list_score_stimR_test))
print('-----------------------------')
print(sum(list_score_stimR_train)/n_neuron)
print(sum(list_score_stimR_test)/n_neuron)

0.9926470588235294
1.0
-----------------------------
0.4632352941176471
0.27941176470588236
-----------------------------
0.5896574245744137
0.5008848811730992


(1) here can index the neuron num which get high score for L/R stimulus and compare whether they are same or related

(2) R score > L score, this is same as spks plot in tutorial, right stimulus seems to get better spike response

## Logistic reg for spks_frag_response---response_move

Investigate the relationship between spike fragment which related to response and mice's movement (either response is a movement)

In [ ]:

list_score_move_train = []
list_score_move_test = []
for idx_neuron in range(n_neuron):
    X_move = spks_frag_response[idx_neuron]  # [n_trial, n_time] (340, 250)
    y_move = response_move  # [n_trial,] (340,)

    X_move_train, X_move_test, y_move_train, y_move_test = train_test_split(X_move, y_move, test_size=0.2)
    log_reg_l1move = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_move_train, y_move_train)

    score_train_move = log_reg_l1move.score(X_move_train, y_move_train)
    score_test_move = log_reg_l1move.score(X_move_test, y_move_test)
    list_score_move_train.append(score_train_move)
    list_score_move_test.append(score_test_move)

In [ ]:
print(max(list_score_move_train))
print(max(list_score_move_test))
print('-----------------------------')
print(min(list_score_move_train))
print(min(list_score_move_test))
print('-----------------------------')
print(sum(list_score_move_train)/n_neuron)
print(sum(list_score_move_test)/n_neuron)

0.8455882352941176
0.9264705882352942
-----------------------------
0.19117647058823528
0.17647058823529413
-----------------------------
0.8105511545592459
0.8129740434855869


different neurons have strongly different relations, reasonable

## Logistic reg for spks_frag_response---response
Only investigate the relationship between spike fragment which related to response and mice's response (i.e. L or R (1/-1, no 0))

In [ ]:
# data prepare
idx_response_LR = np.argwhere(response!=0).squeeze()
response_LR = response[idx_response_LR]  # only take effective response trials [m_trial,] (276,)

spks_frag_response_LR = spks_frag_response[:,idx_response_LR,:]  # [n_neuron, m_trials, n_time] (698, 276, 5)

In [ ]:
# print(idx_response_LR.shape)
# print(response_LR.shape)
# print(response_LR)
# print(spks_frag_response_LR.shape)

In [ ]:

list_score_responseLR_train = []
list_score_responseLR_test = []
for idx_neuron in range(n_neuron):
    X_responseLR = spks_frag_response_LR[idx_neuron]  # [m_trial, n_time] (276, 250)
    y_responseLR = response_LR  # [m_trial,] (276,)

    X_responseLR_train, X_responseLR_test, y_responseLR_train, y_responseLR_test = train_test_split(X_responseLR, y_responseLR, test_size=0.2)
    log_reg_l1responseLR = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_responseLR_train, y_responseLR_train)

    score_train_responseLR = log_reg_l1responseLR.score(X_responseLR_train, y_responseLR_train)
    score_test_responseLR = log_reg_l1responseLR.score(X_responseLR_test, y_responseLR_test)
    list_score_responseLR_train.append(score_train_responseLR)
    list_score_responseLR_test.append(score_test_responseLR)

In [ ]:
print(max(list_score_responseLR_train))
print(max(list_score_responseLR_test))
print('-----------------------------')
print(min(list_score_responseLR_train))
print(min(list_score_responseLR_test))
print('-----------------------------')
print(sum(list_score_responseLR_train)/n_neuron)
print(sum(list_score_responseLR_test)/n_neuron)

0.6727272727272727
0.6785714285714286
-----------------------------
0.44545454545454544
0.32142857142857145
-----------------------------
0.5298515238343313
0.4792263610315188


# TODO
get some visulations; try compact spks into PCA...